In [ ]:
import sys
import os

from yahoo_finance_api2 import share
from yahoo_finance_api2.exceptions import YahooFinanceError
import pandas as pd
import pickle

import prophet
prophet.__version__

In [ ]:
SECURITIES_CODE = "7974.T"

In [ ]:
def data_shaping(symbol_data):
    '''
    yahoo_finance_api2のdataをDataFrameに整形
    '''
    
    # DataFrameに初期化
    df_symbol_data = pd.DataFrame(symbol_data)

    # UNIX時間をUTC時間に変換
    df_symbol_data.timestamp = pd.to_datetime(df_symbol_data.timestamp, unit='ms')

    return df_symbol_data

In [ ]:
def create_sma(df):
    # 終値の単純移動平均(Simple Moving Average:SMA)作成
    rolling_days = [5, 25, 75, 100, 200]
    for rolling_day in rolling_days:
        df["close_sma_"+str(rolling_day)] = df["close"].rolling(rolling_day).mean()

    return df

def create_macd(df):
    # MACD
    # MACD＝短期EMA(指数平滑移動平均)－長期EMA
    # Pandasのewmは厳密に定義と違うらしい
    # https://turtlechan.hatenablog.com/entry/2019/08/15/195048
    df["close_ema_12"] = df["close"].ewm(span=12, adjust=False).mean()
    df["close_ema_26"] = df["close"].ewm(span=26, adjust=False).mean()
    df["close_macd_12_26"] = df["close_ema_12"] - df["close_ema_26"]

    df = df.drop(['close_ema_12', 'close_ema_26'], axis=1)

    return df

def create_rsi(df):
    # RSI(相対力指数)
    # RS =(n日間の終値の上昇幅の平均)/(n日間の終値の下落幅の平均)
    # RSI = 100-(100/(RS+1))
    # n=14で算出

    # 1行前との差分
    df_diff = df['close'].diff()
    # deep copy
    df_up = df_diff.copy(deep=True)
    df_down = df_diff.copy(deep=True)
    
    df_up[df_up < 0] = 0
    df_down[df_down > 0] = 0

    df_up_sum = df_up.rolling(14).sum()
    df_down_sum = df_down.abs().rolling(14).sum()

    df["close_rsi_14"] = df_up_sum/(df_up_sum + df_down_sum) * 100

    # 不要なデータ削除
    del df_diff, df_up, df_down, df_up_sum, df_down_sum
    
    return df

def create_stochas(df):
    # ファーストストキャスティックス
    # n=14, m=3
    df['%K'] = (
        (df['close'] - df['low'].rolling(window=14, center=False).min()) 
        / 
        ( df['high'].rolling(window=14,center=False).max() - df['low'].rolling(window=14,center=False).min())
        ) * 100
    df['%D'] = df['%K'].rolling(window=3,center=False).mean()

    # スローストキャスティクス
    # x=3
    df['Slow%D'] = df['%D'].rolling(window=3, center=False).mean()
    return df


In [ ]:
my_share = share.Share(SECURITIES_CODE) # 東京証券の証券コード指定:任天堂

symbol_data = None

try:
    # 10年分を1日間隔で取得
    symbol_data = my_share.get_historical(share.PERIOD_TYPE_YEAR,
    10,
    share.FREQUENCY_TYPE_DAY,
    1)
except YahooFinanceError as e:
    print(e.message)
    sys.exit(1)

# Dataframeに整形
df = data_shaping(symbol_data)

# 取引時間中に実行すると半端なデータが取れるので
# 前日までのデータを取得
df = df[:-1]

In [ ]:
df = create_sma(df)
df = create_macd(df)
df = create_rsi(df)
df = create_stochas(df)
df = df.dropna(how='any')
df

In [ ]:
df = df.rename(columns={'close': 'y','timestamp':'ds'})
df

In [ ]:
model = prophet.Prophet()
model.fit(df)

future = model.make_future_dataframe(periods=30)
forecast = model.predict(future)
fig = model.plot(forecast.tail(1000))